# LDA - AdaBoost Pipeline 

## Import necessary packages

In [1]:
import time
import optuna
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import AdaBoostClassifier

from datasets import load_dataset

## Constants

In [2]:
RANDOM_SEED = 0

## Load the Hate Speech Filipino dataset from Hugging Face

In [3]:
dataset = load_dataset("hate_speech_filipino")

In [4]:
train_set = dataset["train"]
validation_set = dataset["validation"]
test_set = dataset["test"]

X_train, Y_train = train_set["text"], train_set["label"]
X_val, Y_val = validation_set["text"], validation_set["label"]
X_test, Y_test = test_set["text"], test_set["label"]

X = X_train + X_val + X_test
Y = Y_train + Y_val + Y_test

In [5]:
# Re-split the dataset into training, validation, and test sets
X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size=0.2, random_state=RANDOM_SEED)
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=RANDOM_SEED)

## Vectorize the texts to be able to perform LDA

In [6]:
# Define the CountVectorizer
vectorizer = CountVectorizer(max_df=0.95, min_df=2)
X_train_vector = vectorizer.fit_transform(X_train)

## Perform Latent Dirichlet Allocation on the training set

In [7]:
N_TOPICS = 10
print(f"Performing Latent Dirichlet Allocation for {N_TOPICS} topics")
lda = LatentDirichletAllocation(n_components=N_TOPICS, random_state=RANDOM_SEED)
X_train_topics = lda.fit_transform(X_train_vector)
print(f"Done performing Latent Dirichlet Allocation for {N_TOPICS} topics")

Performing Latent Dirichlet Allocation for 10 topics
Done performing Latent Dirichlet Allocation for 10 topics


In [8]:
# Transform validation and test data using the fitted vectorizer and LDA
X_val_counts = vectorizer.transform(X_val)
X_val_topics = lda.transform(X_val_counts)

X_test_counts = vectorizer.transform(X_test)
X_test_topics = lda.transform(X_test_counts)

## Train an AdaBoost model then evaluate its performance

In [14]:
# Define the objective function for Optuna
def create_objective(X_train, Y_train, X_test, Y_test):
    def objective(trial):
        n_estimators = trial.suggest_int("n_estimators", 100, 1000)
        learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2)
        ada = AdaBoostClassifier(n_estimators=n_estimators, learning_rate=learning_rate, algorithm="SAMME", random_state=RANDOM_SEED)
        ada.fit(X_train, Y_train)
        Y_pred = ada.predict(X_test)
        score = accuracy_score(Y_test, Y_pred)
        return score
    return objective

In [15]:
# Create a study object
study = optuna.create_study(direction="maximize", study_name=f"LDA_AdaBoost_Pipeline")

# Create the study objective
objective = create_objective(X_train_topics, Y_train, X_val_topics, Y_val)

# Execute an optimization
study.optimize(objective, n_trials=20, n_jobs=-1)

# Print the best trial results
print(f"Best Accuracy: {study.best_trial.value}")
print(f"Best hyperparameters: {study.best_trial.params}")

[I 2024-04-30 00:45:31,563] A new study created in memory with name: LDA_AdaBoost_Pipeline
[I 2024-04-30 00:45:35,007] Trial 5 finished with value: 0.5480808914568717 and parameters: {'n_estimators': 123, 'learning_rate': 0.008306560309789691}. Best is trial 5 with value: 0.5480808914568717.
[I 2024-04-30 00:45:36,228] Trial 0 finished with value: 0.529508873297565 and parameters: {'n_estimators': 167, 'learning_rate': 0.0002554080913481626}. Best is trial 5 with value: 0.5480808914568717.
[I 2024-04-30 00:45:38,137] Trial 3 finished with value: 0.602971522905489 and parameters: {'n_estimators': 239, 'learning_rate': 0.009396362137001157}. Best is trial 3 with value: 0.602971522905489.
[I 2024-04-30 00:45:40,598] Trial 10 finished with value: 0.5480808914568717 and parameters: {'n_estimators': 326, 'learning_rate': 0.004993200234210465}. Best is trial 3 with value: 0.602971522905489.
[I 2024-04-30 00:45:40,619] Trial 4 finished with value: 0.6075113495666529 and parameters: {'n_estimat

Best Accuracy: 0.6343375980189847
Best hyperparameters: {'n_estimators': 580, 'learning_rate': 0.007575917059414426}


In [16]:
best_n_estimators = study.best_trial.params["n_estimators"]
best_learning_rate = study.best_trial.params["learning_rate"]

ada = AdaBoostClassifier(n_estimators=best_n_estimators, 
                         learning_rate=best_learning_rate,
                         algorithm="SAMME",
                         random_state=RANDOM_SEED)
ada.fit(X_train_topics, Y_train)
Y_pred = ada.predict(X_test_topics)
score = accuracy_score(Y_test, Y_pred)
print(f"Best LDA-RF Model Accuracy: {score}")

Best LDA-RF Model Accuracy: 0.6307755775577558


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2)
X_train_vector = tfidf_vectorizer.fit_transform(X_train)

In [7]:
from sklearn.decomposition import NMF

N_TOPICS = 10
print(f"Performing Non-negative Matrix Factorization for {N_TOPICS} topics")
nmf = NMF(n_components=N_TOPICS, random_state=RANDOM_SEED)
X_train_topics = nmf.fit_transform(X_train_vector)
print(f"Done performing Non-negative Matrix Factorization for {N_TOPICS} topics")

Performing Non-negative Matrix Factorization for 10 topics
Done performing Non-negative Matrix Factorization for 10 topics


In [8]:
# Transform validation and test data using the fitted vectorizer and LDA
X_val_counts = tfidf_vectorizer.transform(X_val)
X_val_topics = nmf.transform(X_val_counts)

X_test_counts = tfidf_vectorizer.transform(X_test)
X_test_topics = nmf.transform(X_test_counts)

In [9]:
ada = AdaBoostClassifier(n_estimators=948, 
                         learning_rate=0.006593048700236047,
                         algorithm="SAMME",
                         random_state=RANDOM_SEED)
ada.fit(X_train_topics, Y_train)
Y_pred = ada.predict(X_test_topics)
score = accuracy_score(Y_test, Y_pred)
print(f"Best LDA-RF Model Accuracy: {score}")

Best LDA-RF Model Accuracy: 0.6567656765676567
